# Generic playspace

In [1]:
from catalog_manager import esmcat, dfcat, translators

/g/data/tm70/ds0092/software/mambaforge/envs/catalog-manager-dev/lib/python3.10/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [2]:
dfcat_dir = "/g/data/tm70/ds0092/projects/nri_intake_catalog/catalogs"
dfcat_name = "dfcat"

## Add existing CMIP6 esm catalog

In [3]:
%%time

manager = dfcat.CatalogManager.load_esm(
    "/g/data/hh5/public/apps/nci-intake-catalogue/esgf/cmip6/catalogue_latest.json",
    translators.Cmip6Translator
)
manager.add(
    name=dfcat_name,
    directory=dfcat_dir
)

CPU times: user 19.5 s, sys: 16.8 s, total: 36.3 s
Wall time: 37.1 s


## Add new esm catalogs from a config file

In [3]:
import yaml

config = "/g/data/tm70/ds0092/projects/nri_intake_catalog/config/accessesm15.yaml"

with open(config) as f:
    config = yaml.safe_load(f)
    
catalogs = config.get("catalogs")
builder = getattr(esmcat, config.get("builder"))
translator = getattr(translators, config.get("translator"))
esmcat_dir = config.get("catalog_dir")

In [4]:
%%time

for name, kwargs in catalogs.items():
    description = kwargs.pop("description")
    paths = kwargs.pop("paths")
    
    manager = dfcat.CatalogManager.build_esm(
        name=name,
        description=description,
        builder=builder,
        translator=translator,
        paths=paths,
        builder_kwargs=kwargs,
        directory=esmcat_dir,
        overwrite=True,
    )
    manager.add(
        name=dfcat_name,
        directory=dfcat_dir
    )

Successfully wrote ESM catalog json file to: file:///g/data/tm70/ds0092/projects/nri_intake_catalog/catalogs/accessesm15/PI_GWL_B2035.json
CPU times: user 1min 6s, sys: 6.72 s, total: 1min 13s
Wall time: 6min 40s


## Use the DF catalog

In [5]:
import os
import ast
import intake
import pandas as pd

In [6]:
cat = intake.open_df_catalog(
    os.path.join(dfcat_dir, f"{dfcat_name}.csv"),
    name_column="subcatalog",
    columns_with_iterables=["variable"],
)

cat

,description,model,realm,frequency,variable
subcatalog,,,,,
01deg_jra55v13_ryf9091,0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991),ACCESS-OM2,"[ice, ocean]","[1day, 1mon, 3hr, 3mon, fx]","[ANGLE, ANGLET, HTE, HTN, Tsfc_m, age_global, aice_m, aicen_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, average_DT, average_T1, average_T2, bih_fric_u, bih_fric_v, blkmask, ..."
01deg_jra55v140_iaf,Cycle 1/4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[1day, 1mon, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, Tsfc_m, age_global, aice, aice_m, aicen, aicen_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, average_DT, average_T1, average_T2, bih_fric_u, bi..."
01deg_jra55v140_iaf_cycle2,Cycle 2/4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[1day, 1mon, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, Sinz, Sinz_m, Tinz, Tinz_m, Tsfc_m, age_global, aice, aice_m, aicen, aicen_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, average_DT, average_T1..."
01deg_jra55v140_iaf_cycle3,Cycle 3/4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[1day, 1mon, fx]","[ANGLE, ANGLET, HTE, HTN, Sinz, Sinz_m, Tinz, Tinz_m, Tsfc_m, age_global, aice, aice_m, aicen_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, average_DT, average_T1, average_T2,..."
01deg_jra55v140_iaf_cycle4,Cycle 4/4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[1day, 1mon, 3hr, 6hr, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, PP_net, PP_net_m, Sinz, Sinz_m, Tinz, Tinz_m, Tsfc_m, VGRDi, adic, adic_int100, adic_intmld, adic_xflux_adv, adic_yflux_adv, adic_zflux_adv, age_global, aice, aice_..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,Extensions of cycle 4/4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[0hr, 1day, 1mon, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, PP_net, PP_net_m, Tsfc_m, adic, adic_int100, adic_intmld, adic_xflux_adv, adic_yflux_adv, adic_zflux_adv, age_global, aice, aice_m, aicen_m, albsni, albsni_m, algal..."
01deg_jra55v150_iaf_cycle1,Cycle 1/1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0 OMIP2 interannual forcing,ACCESS-OM2,"[ice, ocean]","[1day, 1mon, fx]","[ULAT, ULON, age_global, aice, aice_m, area_t, area_u, average_DT, average_T1, average_T2, blkmask, bmf_u, bmf_v, diff_cbt_t, drag_coeff, dxt, dxu, dyt, dyu, dzt, ekman_we, eta_nonbouss, eta_t, ev..."
025deg_jra55_iaf_omip2_cycle1,Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019),ACCESS-OM2,"[ice, ocean]","[1day, 1mon, 1yr, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, Tair_m, Tsfc_m, age_global, agm, aice, aice_m, aicen_m, aiso_bih, albice_m, albsni_m, albsno_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, ared..."
025deg_jra55_iaf_omip2_cycle2,Cycle 2/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019),ACCESS-OM2,"[ice, ocean]","[1day, 1mon, 1yr, fx]","[ANGLE, ANGLET, HTE, HTN, NCAT, Tair_m, Tsfc_m, age_global, agm, aice, aice_m, aicen_m, aiso_bih, albice_m, albsni_m, albsno_m, alidf_ai_m, alidr_ai_m, alvdf_ai_m, alvdr_ai_m, area_t, area_u, ared..."


In [7]:
cat2 = cat.search(variable="burntFractionAll")

cat2

,description,model,realm,frequency,variable
subcatalog,,,,,
CMIP6_CMS,Available CMIP6 replicas indexed by CLEX-CMS,EC-Earth3-Veg,land,1mon,"[baresoilFrac, burntFractionAll, cCwd, cLeaf, cLitter, cLitterAbove, cLitterBelow, cProduct, cRoot, cSoilFast, cSoilMedium, cSoilSlow, cVeg, cropFrac, evspsblsoi, evspsblveg, fFire, fGrazing, fHar..."


In [8]:
subcat_cmip = cat2.to_subcatalog()

In [9]:
subcat_cmip

,unique
project,1
activity_id,15
institution_id,37
source_id,104
experiment_id,89
member_id,6433
table_id,30
variable_id,450
grid_label,11
date_range,29516
